<h1 align="center"><font color="yellow">Postgres pgvector Extension - Vector Database with PostgreSQL / Langchain Integration</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Link de estudo:

* [Extensão Postgres pgvector - Banco de dados de vetores com integração PostgreSQL / Langchain](https://www.youtube.com/watch?v=FDBnyJu_Ndg)

In [1]:
%pip show langchain

Name: langchain
Version: 0.0.260
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /home/eddygiusepe/1_Eddy_Giusepe/3_estudando_LLMs/Large_Language_Models_LLMs/venv_LLMs/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [11]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings



loader = TextLoader('./RESUMINHO_estado_da_Uniao.txt', encoding='utf-8')
documents = loader.load()

print(documents)  # Imprime os objetos do documento
print(len(documents))  # 1 - lemos apenas um arquivo/documento no carregador


[Document(page_content='Senhora Presidente, Senhora Vice-Presidente, nossa Primeira Dama e Segundo Cavalheiro. Membros do Congresso e do Gabinete. Ministros do Supremo Tribunal Federal. Meus companheiros americanos.\n\nNo ano passado, a COVID-19 nos separou. Este ano finalmente estamos juntos novamente.\n\nHoje à noite, nos encontramos como democratas, republicanos e independentes. Mas o mais importante como americanos.\n\nCom um dever para com o outro, para com o povo americano, para com a Constituição.\n\nE com uma determinação inabalável de que a liberdade sempre triunfará sobre a tirania.\n\nSeis dias atrás, o russo Vladimir Putin procurou abalar as fundações do mundo livre pensando que poderia fazê-lo se curvar a seus modos ameaçadores. Mas ele calculou mal.\n\nEle pensou que poderia entrar na Ucrânia e o mundo viraria. Em vez disso, ele encontrou uma parede de força que nunca imaginou.\n\nEle conheceu o povo ucraniano.\n\nDo presidente Zelenskyy a todos os ucranianos, seu destemo

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20) # Dividimos o texto em chunks
texts = text_splitter.split_documents(documents)

print(texts)
print(len(texts))


[Document(page_content='Senhora Presidente, Senhora Vice-Presidente, nossa Primeira Dama e Segundo Cavalheiro. Membros do Congresso e do Gabinete. Ministros do Supremo Tribunal Federal. Meus companheiros americanos.\n\nNo ano passado, a COVID-19 nos separou. Este ano finalmente estamos juntos novamente.\n\nHoje à noite, nos encontramos como democratas, republicanos e independentes. Mas o mais importante como americanos.\n\nCom um dever para com o outro, para com o povo americano, para com a Constituição.\n\nE com uma determinação inabalável de que a liberdade sempre triunfará sobre a tirania.\n\nSeis dias atrás, o russo Vladimir Putin procurou abalar as fundações do mundo livre pensando que poderia fazê-lo se curvar a seus modos ameaçadores. Mas ele calculou mal.\n\nEle pensou que poderia entrar na Ucrânia e o mundo viraria. Em vez disso, ele encontrou uma parede de força que nunca imaginou.\n\nEle conheceu o povo ucraniano.', metadata={'source': './RESUMINHO_estado_da_Uniao.txt'}), Do

In [13]:
print(texts[0]) # Visualizamos o primeiro chunk


page_content='Senhora Presidente, Senhora Vice-Presidente, nossa Primeira Dama e Segundo Cavalheiro. Membros do Congresso e do Gabinete. Ministros do Supremo Tribunal Federal. Meus companheiros americanos.\n\nNo ano passado, a COVID-19 nos separou. Este ano finalmente estamos juntos novamente.\n\nHoje à noite, nos encontramos como democratas, republicanos e independentes. Mas o mais importante como americanos.\n\nCom um dever para com o outro, para com o povo americano, para com a Constituição.\n\nE com uma determinação inabalável de que a liberdade sempre triunfará sobre a tirania.\n\nSeis dias atrás, o russo Vladimir Putin procurou abalar as fundações do mundo livre pensando que poderia fazê-lo se curvar a seus modos ameaçadores. Mas ele calculou mal.\n\nEle pensou que poderia entrar na Ucrânia e o mundo viraria. Em vez disso, ele encontrou uma parede de força que nunca imaginou.\n\nEle conheceu o povo ucraniano.' metadata={'source': './RESUMINHO_estado_da_Uniao.txt'}


<font color="orange">Agora, vamos converter nossos chunks em embeddings (vetores). </font>

In [14]:
embeddings = OpenAIEmbeddings()

vector = embeddings.embed_query('Testando o modelo Embeddings.')

print(len(vector))  # 1536 dimensions

1536
